In [3]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [1]:
from ultralytics import YOLO

In [2]:
import ultralytics

In [3]:
ultralytics.checks()

Ultralytics 8.3.107 🚀 Python-3.10.15 torch-2.6.0 CPU (Apple M1)
Setup complete ✅ (8 CPUs, 8.0 GB RAM, 164.3/228.3 GB disk)


In [9]:
pip install albumentations


  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 2.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.5 MB/s eta 0:00:0000:0100:01
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import cv2
import glob
import albumentations as A

# Paths
image_dir = 'images'
label_dir = 'label'
output_img_dir = 'images_aug'
output_label_dir = 'labels_aug'

# Augmentation pipeline
transform = A.Compose([
    A.Resize(640, 640),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(0.1, 0.1, p=0.5)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Conversion functions
def yolo_to_voc(x_center, y_center, w, h, img_w, img_h):
    x_min = int((x_center - w / 2) * img_w)
    y_min = int((y_center - h / 2) * img_h)
    x_max = int((x_center + w / 2) * img_w)
    y_max = int((y_center + h / 2) * img_h)
    return x_min, y_min, x_max, y_max

def voc_to_yolo(x_min, y_min, x_max, y_max, img_w, img_h):
    x_center = ((x_min + x_max) / 2) / img_w
    y_center = ((y_min + y_max) / 2) / img_h
    w = (x_max - x_min) / img_w
    h = (y_max - y_min) / img_h
    return x_center, y_center, w, h

# Process all images
for img_path in glob.glob(os.path.join(image_dir, '*.jpg')):
    filename = os.path.basename(img_path).split('.')[0]
    label_path = os.path.join(label_dir, filename + '.txt')

    image = cv2.imread(img_path)
    img_h, img_w = image.shape[:2]

    # Skip if label file doesn't exist
    if not os.path.exists(label_path):
        continue

    # Read YOLO labels
    with open(label_path, 'r') as f:
        yolo_bboxes = []
        class_labels = []
        for line in f:
            parts = line.strip().split()
            cls = int(parts[0])
            x_c, y_c, w, h = map(float, parts[1:])
            x_min, y_min, x_max, y_max = yolo_to_voc(x_c, y_c, w, h, img_w, img_h)
            yolo_bboxes.append([x_min, y_min, x_max, y_max])
            class_labels.append(cls)

    # Apply augmentation
    augmented = transform(image=image, bboxes=yolo_bboxes, class_labels=class_labels)
    aug_img = augmented['image']
    aug_bboxes = augmented['bboxes']
    aug_labels = augmented['class_labels']

    # Save augmented image
    aug_img_name = filename + '_aug.jpg'
    cv2.imwrite(os.path.join(output_img_dir, aug_img_name), aug_img)

    # Convert bboxes back to YOLO and save labels
    aug_img_h, aug_img_w = aug_img.shape[:2]
    with open(os.path.join(output_label_dir, filename + '_aug.txt'), 'w') as f:
        for bbox, cls in zip(aug_bboxes, aug_labels):
            x_min, y_min, x_max, y_max = bbox
            x_c, y_c, w, h = voc_to_yolo(x_min, y_min, x_max, y_max, aug_img_w, aug_img_h)
            f.write(f"{cls} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")


In [ ]:
import cv2
import os

# Set your augmented image and label folder paths
image_dir = 'images_aug'
label_dir = 'labels_aug'
class_names = ['spill']  # update with your actual class list if needed

def draw_yolo_box(image_path, label_path):
    img = cv2.imread(image_path)
    h, w, _ = img.shape

    with open(label_path, 'r') as f:
        for line in f:
            class_id, x_center, y_center, bw, bh = map(float, line.strip().split())
            x1 = int((x_center - bw / 2) * w)
            y1 = int((y_center - bh / 2) * h)
            x2 = int((x_center + bw / 2) * w)
            y2 = int((y_center + bh / 2) * h)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, class_names[int(class_id)], (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, (0, 255, 0), 1)

    cv2.imshow('Check Augmentation', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Loop through and check a few examples
for file in os.listdir(image_dir):
    if file.endswith('.jpg') or file.endswith('.png'):
        img_path = os.path.join(image_dir, file)
        label_path = os.path.join(label_dir, file.replace('.jpg', '.txt').replace('.png', '.txt'))
        if os.path.exists(label_path):
            draw_yolo_box(img_path, label_path)


In [ ]:
import cv2
import os

# Paths
image_dir = 'images_aug'
label_dir = 'labels_aug'
output_dir = 'checked_images'  # folder to save images with boxes

# Create output folder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Class names (change as needed)
class_names = ['spill']

def draw_yolo_box_and_save(image_path, label_path, save_path):
    img = cv2.imread(image_path)
    h, w, _ = img.shape

    with open(label_path, 'r') as f:
        for line in f:
            class_id, x_center, y_center, bw, bh = map(float, line.strip().split())
            x1 = int((x_center - bw / 2) * w)
            y1 = int((y_center - bh / 2) * h)
            x2 = int((x_center + bw / 2) * w)
            y2 = int((y_center + bh / 2) * h)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, class_names[int(class_id)], (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 1)

    # Save the image with boxes
    cv2.imwrite(save_path, img)

# Process and save all images with bounding boxes
for file in os.listdir(image_dir):
    if file.endswith('.jpg') or file.endswith('.png'):
        img_path = os.path.join(image_dir, file)
        label_path = os.path.join(label_dir, file.replace('.jpg', '.txt').replace('.png', '.txt'))

        if os.path.exists(label_path):
            save_path = os.path.join(output_dir, file)
            draw_yolo_box_and_save(img_path, label_path, save_path)


In [ ]:
import os
import shutil
import random

# Set paths
base_path = 'spill_detection'
image_dir = os.path.join(base_path, 'tot_images')
label_dir = os.path.join(base_path, 'tot_labels')

# Target folders
train_img_dir = os.path.join(base_path, 'images/train')
val_img_dir   = os.path.join(base_path, 'images/val')
train_lbl_dir = os.path.join(base_path, 'labels/train')
val_lbl_dir   = os.path.join(base_path, 'labels/val')

# Make sure target dirs exist
for d in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
    os.makedirs(d, exist_ok=True)

# List image files
all_images = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
random.shuffle(all_images)

# 80/20 split
split_index = int(len(all_images) * 0.8)
train_imgs = all_images[:split_index]
val_imgs   = all_images[split_index:]

# Copy function
def copy_files(img_list, target_img_dir, target_lbl_dir):
    for img_file in img_list:
        img_path = os.path.join(image_dir, img_file)
        lbl_file = img_file.rsplit('.', 1)[0] + '.txt'
        lbl_path = os.path.join(label_dir, lbl_file)

        shutil.copy2(img_path, target_img_dir)
        if os.path.exists(lbl_path):
            shutil.copy2(lbl_path, target_lbl_dir)

# Copy to respective folders
copy_files(train_imgs, train_img_dir, train_lbl_dir)
copy_files(val_imgs,   val_img_dir,   val_lbl_dir)

print(f"✅ Split complete!")
print(f"Training images: {len(train_imgs)}")
print(f"Validation images: {len(val_imgs)}")


✅ Split complete!
Training images: 61
Validation images: 16
